# About: Moodle環境の削除--Azure仮想マシンの削除

---

Azureの仮想マシンを削除します。

## パラメータの設定

仮想マシンを削除するのに必要となるパラメータを設定します。

### Azureの認証情報の設定

Azureの操作を行うために必要とサービスプリンシパルに関する情報を指定します。

サブスクリプションIDの値を入力してください。

In [ ]:
from getpass import getpass
azure_subscription_id = getpass()

テナントIDを入力してください。

In [ ]:
azure_tenant = getpass()

クライアントIDを入力してください。

In [ ]:
azure_client_id = getpass()

シークレットを入力してください。

In [ ]:
azure_secret = getpass()

このNotebookから実行するコマンドでAzureのサービスプリンシパルを参照できるようにするために、[環境変数(`AZURE_SUBSCRIPTION_ID`, `AZURE_TENANT`, `AZURE_CLIENT_ID`, `AZURE_SECRET`)](https://docs.microsoft.com/ja-jp/azure/developer/ansible/install-on-linux-vm#env-credentials)の設定を行います。

In [ ]:
import os

os.environ['AZURE_SUBSCRIPTION_ID'] = azure_subscription_id
os.environ['AZURE_TENANT'] = azure_tenant
os.environ['AZURE_CLIENT_ID'] = azure_client_id
os.environ['AZURE_SECRET'] = azure_secret

### 削除対象となる仮想マシンの指定

削除対象となる仮想マシンの名前を指定します。

仮想マシンを作成した際の値を確認するために Ansibleの `group_vars` ファイルを読み込みます。

Ansibleで操作を行う際のグループ名を指定してください。

In [ ]:
# (例)
# target_group = 'Moodle'

target_group =

`group_vars`ファイルに記録されている仮想マシンの名前を表示します。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(target_group)
vm_name = gvars['azure_vm_name']
print(vm_name)

`group_vars`ファイルに記録されているリソースグループの値を表示します。

In [ ]:
resource_group = gvars['azure_resource_group']
print(resource_group)

## 仮想マシンの削除

実際にAzureの仮想マシンを削除します。

In [ ]:
import os
os.environ['PATH'] += f':{os.environ["HOME"]}/.local/bin'
!pipenv run ansible localhost -c local -m azure_rm_virtualmachine -a \
    'state=absent resource_group={resource_group} name={vm_name} \
    remove_on_absent=["all_autocreated"]'

## Ansible 設定ファイルの後始末

Ansibleに関する設定ファイルの後始末を行います。

group_vars に記録されているパラメータを無効にするために group_vars ファイルをリネームします。

In [ ]:
!mv group_vars/{target_group}.yml  group_vars/{target_group}.yml.bak

インベントリから対象となるグループを削除します。

In [ ]:
%run scripts/group.py
remove_group_from_inventory_yml(target_group)

# 削除前との差分を表示します。
! ! diff -u inventory.yml.bak inventory.yml